# K邻近算法（KNN）

构造数据集

In [19]:
import numpy as np
import operator as op

group = np.array([[1., 1.1],[1.,1.],[0, 0],[0, 0.1]])
labels = ['A', 'A', 'B', 'B']

K邻近算法：  
对未知类别属性的数据集众的每个点依次执行以下操作:  
1.计算已知类别数据集中的点与当前点之间的距离;   
2.按照距离递增次序排序  
3.选取与当前点距离最小的k个点  
4.确定前k个点所在类别的出现频率  
5.返回前k个点出现频率最高的类别作为当前点的预测分类

In [51]:
def classify0(inX, dataSet, labels, k):
    # 计算已知类别数据集中的点与当前点之间的距离
    dataSetSize = dataSet.shape[0]
    diffMat = np.tile(inX, (dataSetSize, 1)) - dataSet
    sqDiffMat = diffMat ** 2
    print sqDiffMat
    sqDistances = sqDiffMat.sum(axis=1)
    print sqDistances
    distances = sqDistances ** 0.5
    
    # 按照距离递增次序排序
    sortedDistIndicies = distances.argsort()
    print sortedDistIndicies
    classCount = {}
    # 选取与当前点距离最小的k个点
    for i in range(k):
        voteIlabel = labels[sortedDistIndicies[i]]
        # 确定前k个点所在类别的出现频率
        classCount[voteIlabel] = classCount.get(voteIlabel, 0) + 1
    # 回前k个点出现频率最高的类别作为当前点的预测分类
    sortedClassCount = sorted(classCount.iteritems(), key=op.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]
    

In [52]:
classify0([0, 0], group, labels, 4)

[[ 1.    1.21]
 [ 1.    1.  ]
 [ 0.    0.  ]
 [ 0.    0.01]]
[ 2.21  2.    0.    0.01]
[2 3 1 0]


'A'

In [50]:
np.array([1, 3, 2]).argsort()

array([0, 2, 1], dtype=int64)